In [2]:
import tensorflow as tf
import keras
import os
import cv2
import numpy as np
from src.data_generator import MaskedImageDataGenerator
from net.autoencoder import Autoencoder, custom_loss

In [3]:
import sys

isColab = "google.colab" in sys.modules
data_dir = 'collapsed_data'
# this also works:
# isColab = "COLAB_GPU" in os.environ

if isColab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

    data_dir = ("/content/drive/MyDrive/collapsed_data")

In [4]:
def get_masked_data(sub_dir, mask_denom=5,target_size=(256,256)):
    dir = os.path.join(data_dir, sub_dir)
    files = os.listdir(dir)
    x = []
    y = []
    for f in files:
        path = os.path.join(dir, f)
        try:
            img = cv2.imread(path)
            img = cv2.resize(img, target_size)
            y.append(img)
        except:
            print(path)
            continue
    y = np.array(y, dtype=np.float32)
    x = np.copy(y)
    h,w = target_size
    sec_w = w//mask_denom
    # Make middle black
    offset = (mask_denom // 2)
    x[:,:,sec_w * offset:sec_w * (offset + 1),:]=0
    y=y/255
    x = x/255
    return x,y


In [5]:
name = input('enter model name: ')
ae_model = keras.models.load_model(name,custom_objects={'Autoencoder': Autoencoder, 'custom_loss': custom_loss})

2024-05-11 13:57:51.686855: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-05-11 13:57:51.686875: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-11 13:57:51.686879: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-11 13:57:51.686917: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-11 13:57:51.686934: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/pratham/anaconda3/lib/python3.11/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of typ

In [6]:
print('Evaluating model on testing data')
x_test,y_test = get_masked_data('test')
test = MaskedImageDataGenerator(data_dir + '/test', mask_denom=5, target_size=(256, 256), batch_size=50)
ae_model.evaluate(test, batch_size=32)

Evaluating model on testing data


2024-05-11 13:57:55.693280: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 1s 112ms/step - loss: 0.0453 - mean_squared_error: 0.0095 - mean_absolute_error: 0.0606


[0.04529036208987236, 0.009460107423365116, 0.0606461837887764]

In [7]:
x = x_test[:5]
y = y_test[:5]
pred = ae_model.predict(x)
for i in range(4):
    cv2.imshow('im', x[i])
    cv2.waitKeyEx()
    cv2.imshow('truth', y[i])
    cv2.waitKeyEx()
    cv2.imshow('pred', pred[i])
    cv2.waitKeyEx()
   
cv2.destroyAllWindows()
cv2.waitKey(1)

1/1 [==============================] - 0s 93ms/step


-1

In [8]:
x = x_test[:5]
h,w=(256,256)
rec_w = w//5
modified = []
for i in range(4):
    img1 = x[i]
    img2 = x[i+1]
    new_img = img1.copy()
    new_img[:,rec_w*3:rec_w*5,:]=img2[:,rec_w*3:rec_w*5,:]
    modified.append(new_img)
modified = np.array(modified)
preds = ae_model.predict(modified)
for i in range(4):
    mod_img = modified[i]
    cv2.imshow('mod', mod_img)
    cv2.waitKeyEx()
    cv2.imshow('pred', preds[i])
    cv2.waitKeyEx()

1/1 [==============================] - 0s 42ms/step
